In [681]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pandas_profiling
import numpy as np
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from catboost import CatBoostClassifier
import math

считываем трейн и тест датасет и скрепляем

In [61]:
test_test = train.copy()

In [2]:
train = pd.read_csv("ngwl-predict-customer-churn/train/train.csv")
train.head()

,phone_id,order_completed_at,target
0,19843,2020-01,1.0
1,19843,2020-02,0.0
2,19843,2020-03,0.0
3,19843,2020-04,0.0
4,19843,2020-05,0.0


In [3]:
train['isTrain'] = 1

In [187]:
test = pd.read_csv("ngwl-predict-customer-churn/sample_submission.csv", sep=';', skiprows =1, names=['phone_id','target'], header=None)
test['isTrain'] =0
test['order_completed_at'] = "2020-08"
test.head()

,phone_id,target,isTrain,order_completed_at
0,19843,NaN,0,2020-08
1,471287,NaN,0,2020-08
2,342522,NaN,0,2020-08
3,457799,NaN,0,2020-08
4,233778,NaN,0,2020-08


In [630]:
df = pd.concat([train,test], axis = 0)

### Решение
* считаем фичу how_much_time - фича, показывающая, сколько месяцев назад произошло значение "target"  (от последнего месяца "8"-август отнимаем месяц в колонке order_completed_at
* считаем вес weight с помощью формулы экспоненциального затухания от фичи how_much_time
* считаем вес (значимость) таргета weight_target с учетом затухания
* группируем по пользователю и берем последнее среднее значение в скользящем окне рамером 2 месяца - это и будет результат
* для пользователей, для которых полученные значения оказались NaN - результатом считаем единицу, так если по ним нет никакой информации в обучающей выборке, значит их еще не было и скорее всего это новые пользователи, соответственно это не ооток, а приход, т.е. 1

In [588]:
df['how_much_time'] = df['order_completed_at'].apply(lambda x: 8 - int(x[-1:]))
df['weight'] = df.how_much_time.apply(lambda x: math.pow(1.04, -x * np.e))
df['weight_target'] =  df['weight']*df['target']
df.sort_values(['phone_id','order_completed_at'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [589]:

test2 = test.set_index('phone_id').join(df.groupby('phone_id').weight_target.rolling(2).mean().reset_index().groupby('phone_id').weight_target.last(), how='left')

In [590]:
test2.weight_target.fillna(1, inplace=True)

In [ ]:
predict = [1 if x>=0.5 else 0 for x in test2.weight_target.values]
pd.Series(predict).value_counts()

In [ ]:
#запись в файл
test_final = pd.read_csv("ngwl-predict-customer-churn/sample_submission.csv", sep=';')
columns = test_final.columns 
test_final = pd.Series(predict,index=test.phone_id).reset_index()
test_final.columns=columns
test_final.Predicted = test_final.Predicted.astype('int')
test_final.Id = test_final.Id.astype('int')
test_final.to_csv('submission26.csv',  index=False)

### Вывод и пояснения
* Ниже были попробованы преобразования и алгоритмы, которые НЕ принесли лучшего скора.
* Была проведена агрегация признаков:
        - количественые и dummies признаки, например: котегория, в которой был совершен заказ в данный месяц,
        - средние, общие сумма заказов, их количество, была ли скидка или нет и многое другое
        - модели классификации Catboost, Xgboost
  
Это было сделано, но смотреть не обязательно, так как самый хороший результат дали преобразования выше, а эти модели не показали лучшего результата :)

In [ ]:
# addresses
addresses = pd.read_csv('ngwl-predict-customer-churn/misc/addresses.csv')
addresses
df = df.merge(addresses,how='left', on ='phone_id', copy=False)

In [27]:
# shipments
path = "ngwl-predict-customer-churn/shipments/"
shipments = []
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
for file in onlyfiles:
    shipments.append(pd.read_csv(path + file))
shipments = pd.concat(shipments)
shipments.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,674501,598859,565513,3369534,2018-08-27 07:59:07,2020-08-25 08:11:32,2020-08-25 12:00:00,METRO,complete,shipped,Москва,3,0.0,5,courier,0.0,27922,app,ios,2020-08-25 13:31:33
1,2490493,623354,579546,3397584,2018-09-08 05:13:21,2020-07-25 09:16:10,2020-07-25 12:00:00,METRO,complete,shipped,Казань,62,98.0,0,courier,0.0,24285,app,ios,2020-07-25 13:08:47
2,114409,670532,649133,3472921,2018-10-06 07:02:17,2020-07-20 09:54:34,2020-07-20 16:00:00,METRO,complete,shipped,Москва,10,0.0,0,courier,0.0,26463,web,mac,2020-07-20 17:06:13
3,62296,352974,680884,3448021,2018-09-28 08:31:35,2020-08-03 23:20:58,2020-08-04 06:00:00,METRO,complete,shipped,Москва,9,258.0,5,courier,0.0,44131,app,android,2020-08-04 07:14:32
4,87151,784341,683135,3566202,2018-11-18 17:06:18,2020-07-09 21:59:22,2020-07-10 14:00:00,METRO,complete,shipped,Москва,2,0.0,5,courier,-250.0,13866,web,ios,2020-07-10 16:25:37


In [28]:
shipments['order_completed_at'] = shipments['order_completed_at'].apply(lambda x: x[:7])

In [29]:
df = df.merge(shipments,how='left', left_on =['id', 'order_completed_at'],right_on =['ship_address_id','order_completed_at'], copy=False)
df.head()

,phone_id,order_completed_at,target,isTrain,id,user_id,ship_address_id,shipment_id,order_id,order_created_at,...,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,19843,2020-01,1.0,1,8050123,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19843,2020-01,1.0,1,8194250,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19843,2020-01,1.0,1,3064377,810074.0,3064377.0,2163606.0,5919719.0,2020-01-25 19:39:15,...,Москва,11.0,0.0,0.0,courier,-2000.0,2580.0,web,windows,NaN
3,19843,2020-01,1.0,1,3584090,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19843,2020-01,1.0,1,3584562,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#user_profiles
user_profiles = pd.read_csv('ngwl-predict-customer-churn/misc/user_profiles.csv')
user_profiles


,user_id,gender,bdate
0,2226207,NaN,NaN
1,2226207,NaN,NaN
2,2226207,NaN,NaN
3,1684318,male,1987-10-11
4,1684318,male,1987-10-11
...,...,...,...
396815,1253727,NaN,NaN
396816,1253727,NaN,NaN
396817,1253727,NaN,NaN
396818,1253727,NaN,NaN


In [31]:
df = df.merge(user_profiles,how='left', on ='user_id', copy=False)
df.head()

,phone_id,order_completed_at,target,isTrain,id,user_id,ship_address_id,shipment_id,order_id,order_created_at,...,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at,gender,bdate
0,19843,2020-01,1.0,1,8050123,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19843,2020-01,1.0,1,8194250,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19843,2020-01,1.0,1,3064377,810074.0,3064377.0,2163606.0,5919719.0,2020-01-25 19:39:15,...,0.0,0.0,courier,-2000.0,2580.0,web,windows,NaN,NaN,NaN
3,19843,2020-01,1.0,1,3584090,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19843,2020-01,1.0,1,3584562,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
del shipments, addresses, user_profiles

In [35]:
folders=['ngwl-predict-customer-churn/line_items0' + str(x)+"/" for x in list(range(1,9))]
items = []
columns = ['shipment_id', 'master_category_id', 'created_at', 'price', 'discount','item_id', 'quantity', 'product_id']
for folder in folders:
    onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]
    print(folder)
    for file in onlyfiles:
        items.append(pd.read_csv(folder + file, usecols=columns))
items = pd.concat(items)



ngwl-predict-customer-churn/line_items01/
ngwl-predict-customer-churn/line_items02/
ngwl-predict-customer-churn/line_items03/
ngwl-predict-customer-churn/line_items04/
ngwl-predict-customer-churn/line_items05/
ngwl-predict-customer-churn/line_items06/
ngwl-predict-customer-churn/line_items07/
ngwl-predict-customer-churn/line_items08/


In [125]:
items.head()

,shipment_id,created_at,price,discount,item_id,quantity,product_id,id,name,parent_category_id
0,387023,2018-03-02 17:23:48,64.989998,0.0,3857464,1,14840,34,Яйца,20
1,695703,2019-01-23 14:16:51,86.000000,0.0,9558041,1,58795,34,Яйца,20
2,695703,2019-01-23 14:16:59,84.000000,0.0,9558045,1,69372,34,Яйца,20
3,817880,2019-02-21 08:36:55,101.000000,0.0,10464479,1,14836,34,Яйца,20
4,817880,2019-02-21 08:41:53,81.000000,0.0,10464666,1,14838,34,Яйца,20


In [37]:
#categories
categories = pd.read_csv('ngwl-predict-customer-churn/misc/master_categories.csv')
items = items.merge(categories, left_on = 'master_category_id', right_on = "id")



In [38]:
items

items = items.merge(categories, left_on = 'parent_category_id', right_on = "id")
items.drop(['name_x', 'parent_category_id_x'], axis = 1, inplace=True)

items = items.merge(categories, left_on = 'parent_category_id_y', right_on = "id")
items.drop(['parent_category_id', 'name_y','id_x','parent_category_id_y','id_y'], axis = 1, inplace=True)
items.drop(['created_at','item_id','product_id','id','master_category_id'],axis =1, inplace=True)

items.head()

,shipment_id,price,discount,quantity,name
0,387023,64.989998,0.0,1,Продукты питания
1,695703,86.000000,0.0,1,Продукты питания
2,695703,84.000000,0.0,1,Продукты питания
3,817880,101.000000,0.0,1,Продукты питания
4,817880,81.000000,0.0,1,Продукты питания


In [39]:
items = pd.pivot_table(items, values=['price','quantity'], index='shipment_id', columns=['name']).reset_index()

In [40]:
items.columns = ['shipment_id','priceАвтотовары',
                'priceАптека',
                  'priceБытовая техника',
                  'priceБытовая химия',
                  'priceВсе для ремонта',
                  'priceДача и сад',
                  'priceДетские товары',
                  'priceКанцелярские товары',
                  'priceКосметика и гигиена',
                  'priceПодарки и украшения',
                  'priceПродукты питания',
                  'priceСпортивные товары',
                  'priceТовары для дома',
                  'priceТовары для животных',
                  'priceТовары для кухни',
                  'priceТуризм и отдых',
                  'priceЭлектроника',
               'quantityАвтотовары',
               'quantityАптека',
               'quantityБытовая техника',
               'quantityБытовая химия',
               'quantityВсе для ремонта',
               'quantityДача и сад',
               'quantityДетские товары',
               'quantityКанцелярские товары',
               'quantityКосметика и гигиена',
               'quantityПодарки и украшения',
               'quantityПродукты питания',
               'quantityСпортивные товары',
           'quantityТовары для дома',
               'quantityТовары для животных',
               'quantityТовары для кухни',
           'quantityТуризм и отдых',
               'quantityЭлектроника']

In [41]:
items = items.groupby('shipment_id').mean().reset_index()
items.fillna(0, inplace=True)

In [42]:
items['summ'] = pd.Series((items[['priceАвтотовары', 'priceАптека', 'priceБытовая техника',
       'priceБытовая химия', 'priceВсе для ремонта', 'priceДача и сад',
       'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника']].values * items[['quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПродукты питания',
       'quantityСпортивные товары', 'quantityТовары для дома',
       'quantityТовары для животных', 'quantityТовары для кухни',
       'quantityТуризм и отдых', 'quantityЭлектроника']].values).sum(axis = 1), index = items.index)

In [43]:
items

,shipment_id,priceАвтотовары,priceАптека,priceБытовая техника,priceБытовая химия,priceВсе для ремонта,priceДача и сад,priceДетские товары,priceКанцелярские товары,priceКосметика и гигиена,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,178163,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,1.387097,0.0,0.0,0.0,0.0,0.0,0.0,270.779188
1,273988,0.0,0.0,0.0,398.131764,0.0,0.0,0.000000,0.0,266.000000,...,1.00,0.0,1.516129,0.0,0.0,0.0,0.0,0.0,0.0,1314.596115
2,322307,0.0,0.0,0.0,94.900002,0.0,0.0,0.000000,0.0,57.152500,...,3.50,0.0,1.210526,0.0,0.0,0.0,0.0,0.0,0.0,1275.017701
3,337809,0.0,0.0,0.0,104.900002,0.0,0.0,0.000000,0.0,110.500000,...,1.25,0.0,1.725000,0.0,0.0,0.0,0.0,0.0,0.0,464.237277
4,351762,0.0,0.0,0.0,499.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,1.405405,0.0,1.0,1.0,0.0,0.0,0.0,2815.823573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257574,9915183,0.0,0.0,0.0,0.000000,0.0,0.0,33.934444,0.0,99.989998,...,1.00,0.0,1.285714,0.0,0.0,0.0,0.0,0.0,0.0,231.843473
2257575,9915189,0.0,0.0,2199.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,3.777778,0.0,1.0,0.0,0.0,0.0,0.0,2414.125187
2257576,9915206,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,541.400000
2257577,9915207,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,131.990005,...,1.00,0.0,1.500000,0.0,0.0,0.0,0.0,0.0,0.0,237.593756


In [44]:
df = df.merge( items, on='shipment_id', how='left')

In [45]:
del items

In [47]:

df.columns

Index(['phone_id', 'order_completed_at', 'target', 'isTrain', 'id', 'user_id',
       'ship_address_id', 'shipment_id', 'order_id', 'order_created_at',
       'shipment_starts_at', 'retailer', 's.order_state', 'shipment_state',
       's.city_name', 's.store_id', 'total_cost', 'rate', 'dw_kind',
       'promo_total', 'total_weight', 'platform', 'os', 'shipped_at', 'gender',
       'bdate', 'priceАвтотовары', 'priceАптека', 'priceБытовая техника',
       'priceБытовая химия', 'priceВсе для ремонта', 'priceДача и сад',
       'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'q

In [ ]:
full_df = addresses.merge(shipments, left_on ="id", right_on="ship_address_id", copy=False )
del full_df["id"]

full_df = full_df.merge(user_profiles, on="user_id" )
full_df = full_df.merge(items, on="shipment_id")

del addresses, user_profiles, items, shipments

full_df = full_df.merge(categories, left_on="master_category_id", right_on="id" )

In [11]:
full_df["total_product_sum"] = (full_df.price -full_df.discount) * full_df.quantity
full_df["discount_bool"] = (full_df.discount>0).astype('int')

In [12]:
city2index={}
cities = full_df['s.city_name'].value_counts().sort_values(False).index.values
for n,city in enumerate(cities):
    city2index[city] = n
full_df['s.city_name'] = full_df['s.city_name'].map(city2index)

In [16]:
full_df['s.order_state'] = full_df['s.order_state'].map({'complete':1, 'canceled':0,'resumed':1,'cart':0})
full_df['order_completed_at'] = full_df['order_created_at'].apply(lambda x: str(x)[:7])



In [719]:
full_df

,phone_id,order_completed_at,target,isTrain,retailer,s.order_state,shipment_state,s.city_name,total_cost,rate,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,19843,2020-01,1.0,1,2.0,0.0,0.0,43.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1168.450001
3,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170535,23853,2020-08,0.0,0,2.0,0.0,0.0,43.0,158.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1699.000000
14170536,23853,2020-08,0.0,0,26.0,0.0,0.0,86.0,199.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1078.319946
14170537,23853,2020-08,0.0,0,2.0,0.0,2.0,43.0,158.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
14170538,23853,2020-08,0.0,0,2.0,0.0,0.0,43.0,158.0,0.0,...,0.0,0.0,6.5,0.0,0.0,0.0,0.0,1.0,0.0,1237.135006


In [204]:
full_df.fillna(0, inplace=True)
full_df.head()

,phone_id,order_completed_at,target,isTrain,order_created_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,19843,2020-01,1.0,1,0,0,no_retailer,no_order,no_shipment,no_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,19843,2020-01,1.0,1,0,0,no_retailer,no_order,no_shipment,no_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,19843,2020-01,1.0,1,2020-01-25 19:39:15,2020-01-26 20:00:00,METRO,canceled,canceled,Москва,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1168.450001
3,19843,2020-01,1.0,1,0,0,no_retailer,no_order,no_shipment,no_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,19843,2020-01,1.0,1,0,0,no_retailer,no_order,no_shipment,no_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [278]:
full_df['s.order_state'] = full_df['s.order_state'].astype("object")

In [50]:
df.drop(for_drop, axis = 1, inplace=True)

In [52]:
for_drop = ['id', 'user_id',
       'ship_address_id', 'shipment_id', 'order_id']
df.drop(for_drop, axis = 1, inplace=True)
full_df = df

In [53]:
full_df.fillna({'gender':'no_gender',
              'retailer':'no_retailer',
              's.order_state':'no_order',
              'shipment_state':'no_shipment',
              's.city_name':'no_city',
              'total_cost':0,
              'rate':0,
              'dw_kind':'no_dw',
              'promo_total':0,
              'total_weight':0,
              'platform':'no_platform',
              'dw_kind':'no_os'
             }, inplace=True)

In [211]:
full_df.columns

Index(['phone_id', 'order_completed_at', 'target', 'isTrain',
       'order_created_at', 'retailer', 's.order_state', 'shipment_state',
       's.city_name', 'total_cost', 'rate', 'dw_kind', 'promo_total',
       'total_weight', 'platform', 'os', 'shipped_at', 'gender', 'bdate',
       'priceАвтотовары', 'priceАптека', 'priceБытовая техника',
       'priceБытовая химия', 'priceВсе для ремонта', 'priceДача и сад',
       'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quant

In [718]:
full_df

,phone_id,order_completed_at,target,isTrain,retailer,s.order_state,shipment_state,s.city_name,total_cost,rate,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,19843,2020-01,1.0,1,2.0,0.0,0.0,43.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1168.450001
3,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170535,23853,2020-08,0.0,0,2.0,0.0,0.0,43.0,158.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1699.000000
14170536,23853,2020-08,0.0,0,26.0,0.0,0.0,86.0,199.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1078.319946
14170537,23853,2020-08,0.0,0,2.0,0.0,2.0,43.0,158.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
14170538,23853,2020-08,0.0,0,2.0,0.0,0.0,43.0,158.0,0.0,...,0.0,0.0,6.5,0.0,0.0,0.0,0.0,1.0,0.0,1237.135006


In [54]:
#full_df.to_csv('full_df.csv')

In [213]:
cat = ['s.city_name','gender','s.order_state','shipment_state','dw_kind','platform','retailer']

In [777]:
full_df['s.order_state'].value_counts()

3.0    9084861
2.0    4736340
0.0     341160
4.0       8119
1.0         60
Name: s.order_state, dtype: int64

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
        39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
        52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
        65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
        78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
        91., 92., 93., 94., 95., 96., 97.]),
 array([0., 1., 2.]),
 array([0., 1., 2., 3., 4.]),
 array([0., 1., 2., 3., 4., 5., 6., 7.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
        39., 40., 41., 42., 43., 44., 45., 46.])]

In [755]:
##### encoder = OrdinalEncoder()
full_df[cat] = encoder.fit_transform(full_df[cat])

KeyboardInterrupt: 

In [219]:
df[['phone_id','weight_targe']]

,phone_id,order_completed_at,target,isTrain,weight,weight_target
0,19843,7,1.0,1,0.474122,0.474122
1,19843,6,0.0,1,0.527462,0.000000
2,19843,5,0.0,1,0.586804,0.000000
3,19843,4,0.0,1,0.652821,0.000000
4,19843,3,0.0,1,0.726266,0.000000
...,...,...,...,...,...,...
214604,16841,0,NaN,0,1.000000,NaN
214605,69348,0,NaN,0,1.000000,NaN
214606,594974,0,NaN,0,1.000000,NaN
214607,320024,0,NaN,0,1.000000,NaN


In [218]:
full_df.head()

,phone_id,order_completed_at,target,isTrain,retailer,s.order_state,shipment_state,s.city_name,total_cost,rate,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,19843,2020-01,1.0,1,2.0,0.0,0.0,43.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1168.450001
3,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,19843,2020-01,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [735]:
full_df['s.order_state'].value_counts()

3.0    9084861
2.0    4736340
0.0     341160
4.0       8119
1.0         60
Name: s.order_state, dtype: int64

In [778]:
full_df.columns

num_orders = full_df[full_df['s.order_state']==2].groupby(['order_completed_at','phone_id'], as_index=False)[['s.order_state']].count()



In [779]:
num_orders.shape

(845291, 3)

In [226]:
second_agg = full_df.groupby(['phone_id','order_completed_at']).mean().reset_index()

In [229]:
second_agg.columns

Index(['phone_id', 'order_completed_at', 'target', 'isTrain', 'retailer',
       's.order_state', 'shipment_state', 's.city_name', 'total_cost', 'rate',
       'dw_kind', 'promo_total', 'total_weight', 'platform', 'gender',
       'priceАвтотовары', 'priceАптека', 'priceБытовая техника',
       'priceБытовая химия', 'priceВсе для ремонта', 'priceДача и сад',
       'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПродукты питания',
   

In [231]:
second_agg[['target','retailer',
       's.order_state', 'shipment_state', 's.city_name', 'total_cost', 'rate',
       'dw_kind', 'promo_total', 'total_weight', 'platform', 'gender',
       'priceАвтотовары', 'priceАптека', 'priceБытовая техника',
       'priceБытовая химия', 'priceВсе для ремонта', 'priceДача и сад',
       'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПродукты питания',
       'quantityСпортивные товары', 'quantityТовары для дома',
       'quantityТовары для животных', 'quantityТовары для кухни',
       'quantityТуризм и отдых', 'quantityЭлектроника', 'summ']].corr().sort_values('target')

,target,retailer,s.order_state,shipment_state,s.city_name,total_cost,rate,dw_kind,promo_total,total_weight,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
dw_kind,-0.059757,-0.287713,0.940270,-0.954306,-0.874499,-0.759955,-0.706474,1.000000,0.302123,-0.666231,...,-0.316709,-0.042185,-0.267624,-0.036529,-0.265185,-0.133176,-0.150605,-0.081603,-0.121707,-0.467480
s.order_state,-0.056930,-0.265935,1.000000,-0.813364,-0.829077,-0.721038,-0.631974,0.940270,0.291953,-0.630151,...,-0.302189,-0.039736,-0.258829,-0.035743,-0.252803,-0.125374,-0.143549,-0.076781,-0.114905,-0.450492
gender,-0.042060,-0.093167,0.298779,-0.296271,-0.277539,-0.235848,-0.229319,0.311806,0.099719,-0.209113,...,-0.099118,-0.010619,-0.074387,-0.009621,-0.080524,-0.055145,-0.048474,-0.025774,-0.038409,-0.132576
platform,-0.010008,-0.184287,0.384057,-0.373968,-0.359185,-0.285766,-0.483686,0.402728,0.147457,-0.202803,...,-0.103081,-0.021124,-0.096282,-0.025004,-0.102057,-0.044481,-0.059528,-0.028621,-0.027305,-0.182085
priceАвтотовары,-0.002687,-0.009571,-0.044998,0.039666,0.038756,0.031970,0.032948,-0.043932,-0.042144,0.031269,...,0.014167,0.006605,0.008972,0.011314,0.014121,0.005052,0.011399,0.008120,0.012723,0.112841
quantityКанцелярские товары,-0.000650,-0.000724,-0.037218,0.037297,0.035142,0.033395,0.028776,-0.039141,-0.010716,0.035967,...,0.028508,0.010122,0.013876,0.002978,0.028219,0.005758,0.015037,0.010100,0.013837,0.057862
priceВсе для ремонта,0.000306,-0.005256,-0.013038,0.012252,0.010942,0.012125,0.010593,-0.013327,-0.006169,0.011192,...,0.003742,0.000794,0.003233,0.001200,0.007443,0.001513,0.005163,0.003673,0.006463,0.156915
priceПодарки и украшения,0.000356,-0.008688,-0.027693,0.026708,0.023536,0.028041,0.020119,-0.028552,-0.008822,0.026586,...,0.012146,0.202584,0.006588,0.004845,0.015646,0.004479,0.014337,0.008022,0.011524,0.057754
priceТовары для животных,0.000510,0.028805,-0.137796,0.138374,0.127017,0.119650,0.101015,-0.144982,-0.041584,0.135354,...,0.046697,0.006940,0.030529,0.005636,0.047798,0.129895,0.020605,0.015059,0.020943,0.193191
quantityАвтотовары,0.000622,-0.007428,-0.047061,0.044797,0.041258,0.041915,0.034962,-0.048049,-0.027646,0.050888,...,0.026006,0.006520,0.012762,0.005907,0.038145,0.005454,0.015825,0.011899,0.019114,0.083794


In [239]:
second_agg

,phone_id,order_completed_at,target,isTrain,retailer,s.order_state,shipment_state,s.city_name,total_cost,rate,...,quantityКосметика и гигиена,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ
0,2,2020-01,1.0,1,5.2,2.8,2.8,8.6,19.6,0.0,...,0.20,0.0,0.225000,0.0,0.2,0.400000,0.0,0.0,0.0,150.036492
1,2,2020-02,1.0,1,4.4,2.6,3.6,17.2,73.2,0.0,...,0.25,0.0,0.608333,0.0,0.0,0.500000,0.0,0.0,0.0,113.611802
2,2,2020-03,0.0,1,5.2,2.8,2.8,8.6,46.6,0.0,...,0.20,0.0,0.234286,0.0,0.0,0.200000,0.0,0.0,0.0,78.044152
3,2,2020-04,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,2,2020-05,0.0,1,5.2,2.8,2.8,8.6,49.6,0.0,...,0.00,0.0,0.242857,0.0,0.2,0.213333,0.0,0.0,0.0,176.396904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374180,719373,2020-08,0.0,0,2.0,2.0,6.0,44.0,0.0,0.0,...,0.00,0.0,2.363636,0.0,0.0,0.000000,0.0,0.0,0.0,341.161901
1374181,719374,2020-05,0.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1374182,719374,2020-06,0.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1374183,719374,2020-07,1.0,1,6.0,3.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [243]:
second_agg[['phone_id','order_completed_at']]

,phone_id,order_completed_at
0,2,2020-01
1,2,2020-02
2,2,2020-03
3,2,2020-04
4,2,2020-05
...,...,...
1374180,719373,2020-08
1374181,719374,2020-05
1374182,719374,2020-06
1374183,719374,2020-07


In [592]:
#second_agg.drop(['isTrain_x','isTrain_y','weight_target_x','weight_target_y'], axis = 1, inplace=True)
second_agg.drop(['isTrain','weight_target'], axis = 1, inplace=True)

In [593]:
second_agg = df[['phone_id','isTrain','order_completed_at','weight_target']].merge(second_agg, left_on = ['phone_id','order_completed_at'], right_on = ['phone_id','order_completed_at'], how='left')

In [725]:
second_agg.columns

Index(['phone_id', 'isTrain', 'order_completed_at', 'weight_target', 'target',
       'retailer', 's.order_state', 'shipment_state', 's.city_name',
       'total_cost', 'rate', 'dw_kind', 'promo_total', 'total_weight',
       'platform', 'gender', 'priceАвтотовары', 'priceАптека',
       'priceБытовая техника', 'priceБытовая химия', 'priceВсе для ремонта',
       'priceДача и сад', 'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПроду

In [595]:
second_agg.isTrain.value_counts()

1    1159576
0     214609
Name: isTrain, dtype: int64

In [782]:
second_agg = second_agg.merge(num_orders, on =['phone_id','order_completed_at'], how='left')

In [790]:
final_df

,phone_id,isTrain,order_completed_at,weight_target,target,retailer,s.order_state_x,shipment_state,s.city_name,total_cost,...,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ,s.order_state
2,2,1,2020-03,5.007922e-01,0.0,5.2,2.8,2.8,8.6,46.6,...,0.0,0.234286,0.0,0.0,0.200000,0.0,0.0,0.0,78.044152,1.0
3,2,1,2020-04,2.637312e-01,1.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,2,1,2020-05,3.264107e-01,0.0,5.2,2.8,2.8,8.6,49.6,...,0.0,0.242857,0.0,0.2,0.213333,0.0,0.0,0.0,176.396904,1.0
5,2,1,2020-06,3.264107e-01,0.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
6,2,1,2020-07,0.000000e+00,0.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374172,719360,1,2020-04,2.647882e-14,0.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1374173,719360,1,2020-05,2.647882e-14,0.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1374174,719360,1,2020-06,2.647882e-14,0.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1374175,719360,1,2020-07,2.647882e-14,1.0,6.0,3.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


In [791]:
final_df = second_agg.fillna(0)

In [792]:
X, y = final_df.drop('target', axis = 1), final_df['target']
X.order_completed_at = LabelEncoder().fit_transform(X.order_completed_at)

In [793]:
X_train, y_train, X_test = X[X.isTrain==1],y[X.isTrain==1], X[X.isTrain==0] 
X_train.shape, y_train.shape, X_test.shape

((1159576, 51), (1159576,), (214609, 51))

In [794]:
X_test

,phone_id,isTrain,order_completed_at,weight_target,retailer,s.order_state_x,shipment_state,s.city_name,total_cost,rate,...,quantityПодарки и украшения,quantityПродукты питания,quantityСпортивные товары,quantityТовары для дома,quantityТовары для животных,quantityТовары для кухни,quantityТуризм и отдых,quantityЭлектроника,summ,s.order_state
14,3,0,7,0.403987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,9,0,7,0.403987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,10,0,7,0.449437,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69,13,0,7,0.403987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,16,0,7,0.403987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374168,719359,0,7,0.449437,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1374176,719360,0,7,0.449437,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1374178,719363,0,7,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1374180,719373,0,7,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [795]:
X_train_, X_valid, y_train_, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [796]:
X_train.columns

Index(['phone_id', 'isTrain', 'order_completed_at', 'weight_target',
       'retailer', 's.order_state_x', 'shipment_state', 's.city_name',
       'total_cost', 'rate', 'dw_kind', 'promo_total', 'total_weight',
       'platform', 'gender', 'priceАвтотовары', 'priceАптека',
       'priceБытовая техника', 'priceБытовая химия', 'priceВсе для ремонта',
       'priceДача и сад', 'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПродукты пита

In [798]:
from sklearn.metrics import f1_score
cols =[ 'order_completed_at','weight_target',
       'retailer', 's.order_state_x', 'shipment_state', 's.city_name',
       'total_cost', 'rate', 'dw_kind', 'promo_total', 'total_weight',
       'platform', 'gender', 'priceАвтотовары', 'priceАптека',
       'priceБытовая техника', 'priceБытовая химия', 'priceВсе для ремонта',
       'priceДача и сад', 'priceДетские товары', 'priceКанцелярские товары',
       'priceКосметика и гигиена', 'priceПодарки и украшения',
       'priceПродукты питания', 'priceСпортивные товары',
       'priceТовары для дома', 'priceТовары для животных',
       'priceТовары для кухни', 'priceТуризм и отдых', 'priceЭлектроника',
       'quantityАвтотовары', 'quantityАптека', 'quantityБытовая техника',
       'quantityБытовая химия', 'quantityВсе для ремонта',
       'quantityДача и сад', 'quantityДетские товары',
       'quantityКанцелярские товары', 'quantityКосметика и гигиена',
       'quantityПодарки и украшения', 'quantityПродукты питания',
       'quantityСпортивные товары', 'quantityТовары для дома',
       'quantityТовары для животных', 'quantityТовары для кухни',
       'quantityТуризм и отдых', 'quantityЭлектроника', 'summ',
       's.order_state']
model = XGBClassifier()
model.fit(X_train_[cols], y_train_)

predict = model.predict(X_valid[cols])
predict_proba = model.predict_proba(X_valid[cols].fillna(0))
f1_score(y_valid, predict)

0.697185798635074

In [800]:
model.feature_importances_

array([1.0286823e-02, 7.9554981e-03, 3.3346750e-02, 5.0750403e-03,
       2.2280902e-02, 1.0311026e-01, 2.4499029e-03, 1.4210188e-02,
       5.2135473e-01, 1.6318330e-03, 1.4317621e-01, 2.7251136e-03,
       1.9459295e-03, 1.2811817e-03, 9.2898437e-04, 6.6054094e-04,
       9.6455769e-04, 1.5134289e-03, 1.0713093e-03, 1.0046929e-03,
       1.2852912e-03, 1.2632379e-03, 1.1119754e-03, 4.3058554e-03,
       1.2532234e-03, 1.3608331e-03, 9.3915296e-04, 1.4051426e-03,
       1.0651551e-03, 1.0486800e-03, 1.2851364e-03, 3.6112958e-04,
       1.0431345e-03, 1.6711099e-03, 8.0297765e-04, 1.4461735e-03,
       1.0898703e-03, 1.0127942e-03, 1.6316602e-03, 1.0439929e-03,
       1.2046740e-03, 1.3589126e-03, 1.6762437e-03, 1.0034551e-03,
       1.4622376e-03, 8.2057866e-04, 1.1767432e-03, 9.8678377e-04,
       8.6909957e-02], dtype=float32)

In [621]:
for tresh in [0.2,0.4,0.45,0.6]:
    print(f1_score(y_valid, [1 if x>tresh else 0 for x in predict_proba[:,1]]))

0.6122058519590186
0.6912802760950484
0.6870101859958573
0.6548521256931608


In [623]:

model =  XGBClassifier()
model.fit(X_train[cols], y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [515]:
#predict = model.predict(X_test[cols])

predict_proba = model.predict_proba(X_test[cols])
predict = [1 if x>=0.2 else 0 for x in predict_proba[:,1]]
pd.Series(predict).value_counts()

1    214609
dtype: int64

In [714]:
#predict = [1 if x>=0.4 else 0 for x in test2.weight_target.values]
predict = [1 if x>=0.3 else 0 for x in pr]
pd.Series(predict).value_counts()

1    195618
0     18991
dtype: int64

In [715]:
test_final = pd.read_csv("ngwl-predict-customer-churn/sample_submission.csv", sep=';')
columns = test_final.columns 
test_final = pd.Series(predict,index=test.phone_id).reset_index()
test_final.columns=columns
test_final.Predicted = test_final.Predicted.astype('int')
test_final.Id = test_final.Id.astype('int')
test_final.to_csv('submission26.csv',  index=False)

In [152]:
predict

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,


In [127]:
pd.Series(predict,index=X[X.isTrain==0].phone_id).reset_index()

,phone_id,0
0,19843,1.0
1,471287,0.0
2,342522,0.0
3,457799,1.0
4,233778,0.0
...,...,...
214604,16841,1.0
214605,69348,1.0
214606,594974,0.0
214607,320024,0.0


In [103]:
predict = model.predict(X_test[['phone_id','rate', 'price','total_product_sum']])

In [85]:
train_df = full_df[['phone_id', 'order_id', 's.order_state', 's.city_name', 'rate','gender', 'total_product_sum','order_completed_at']]

In [88]:
train_df.groupby()

,phone_id,order_id,s.order_state,s.city_name,rate,total_product_sum,order_completed_at
0,5,207513,1,82,0,344.949982,2016-07
1,6,2954984,1,96,1,1420.000000,2018-01
2,6,2954984,1,96,1,1420.000000,2018-01
3,6,2954984,1,96,1,1420.000000,2018-01
4,6,2954984,1,96,1,1420.000000,2018-01
...,...,...,...,...,...,...,...
89993229,141976,11663579,1,88,3,58.600002,2020-06
89993230,141976,11663579,1,88,3,58.600002,2020-06
89993231,141976,11663579,1,88,3,58.600002,2020-06
89993232,141976,11663579,1,88,3,58.600002,2020-06


In [369]:
s1 = pd.read_csv('submission16.csv', index_col="Id")
s2 = pd.read_csv('submission7.csv', index_col="Id")

In [385]:
pd.DataFrame(pd.Series(s1.Predicted.values & s2.Predicted.values, index=s1.index, name='Predicted')).to_csv('submission21.csv')